In [3]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [1]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [1]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #-------------------------------------------------------------- ok
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #--------------------------------------------------------------- ok
        
    #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    

    #--------------------------------------------------------------- ok

    lista_actores_peli_i=[]
    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,12):

        #accedemos a la informacion del actor/actriz:
        try:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
            sleep(5)                    
            #-----------------------------------------------------------------ok

            #seleccionamos MENU desplegable, para acceder a biografia y premios:
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
            sleep(2)
            #acceso a la biografia
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
            sleep(2)
            #----------------------------------------------------------------ok
            
            # obtenemos el NOMBRE del actor
            nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
            sleep(5)
            #---------------------------------------------------------------- ok
            #obtenemos AÑO
            try:
                año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
                año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
            except:
                año='NULL'
            sleep(3)

            #---------------------------------------------------------------- ok

            #obtenemos el texto para filtrar informacion 'CONOCIDO POR':
            try:
                # scroll hasta el texto que necesitamos
                xpath_d='//*[@id="mini_bio"]'
                pyhton_d= driver.find_element(By.XPATH, xpath_d)
                driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
                sleep(6)
                #Obtenemos el texto:
                driver.find_element("xpath", '//*[@id="mini_bio"]').click()
                texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text
                
                sleep(6)
                conocido_por=[]
                if re.search('conocido por|conocida por', texto_actor) == None:
                        conocido_por='NULL'
                else:
                    patron_a='\.'
                    patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                    patron_c='\(\d{4}'
                    patron_d="\), |\) y |\)"
                    lista_sep=re.split(patron_b, texto_actor)
                    lista_sep_2=re.split(patron_a, lista_sep[1])
                    # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                    lista_sep_3=re.split(patron_c, lista_sep_2[0])
                        
                    for y in lista_sep_3:
                        lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                        try: 
                            lista_sep_3_sin_espacios=re.sub('su papel en ','',lista_sep_3_sin_espacios)
                        except:
                            continue
                        
                        if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                            conocido_por.append(lista_sep_3_sin_espacios)
            except:
                conocido_por='NULL'
            #---------------------------------------------------------------- ok
            sleep(2)
            driver.back() # al acceder a 'mi biografía' hacemos click. tenemos que volver atrás. SEGUIMOS DENTRO DE BIOGRAFIA ACTOR
            sleep(2)
            driver.back() # VOLVEMOS A PAGINA PRINCIPAL ACTOR.
            sleep(10)

            #--------------------------------------------------------------- ok
            #acceso a los PREMIOS, desde el desplegable.
            try:
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
            
            except:
                #si se sale de la pagina, (si no hay texto, conocido por) volvemos a ella:
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click() #acceso actor.
                sleep(2) 
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
                sleep(2)

    
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la premios.
            sleep(6)
            #creamos ciclo for para sacar las los premios:
            
            try:
                premios=[]
                for i in range(1,6): #solo queremos 5 pelis
                    premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                    premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                    premios.append(premio_limpio)
                sleep(6)
            except:
                premios='NULL'
            
            

            driver.back()
            sleep(3)

            #--------------------------------------------------------------- ok
            #extraemos informacion que hace.

            que_hace=[]
            try:
                try:
                    for z in range(1,4):
                        peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                        que_hace.append(peli_hace)
                        sleep(2)
                except:
                        peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                        que_hace.append(peli_hace)
                        sleep(2)
            except:
                que_hace='NULL'
            
            #--------------------------------------------------------------- ok

            #sacamos tupla de cada actor
            lista_infor_actor=[nombre, año, que_hace, conocido_por,premios,id_pelicula]
            tupla_x=tuple(lista_infor_actor)
            lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula
        
            sleep(6)
            driver.back()
            #volvemos a la página principal 
        except:
            pass

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


=========================================================================================

Importamos el csv de las películas para obtener los id_peliculas extraidos:

=========================================================================================

In [3]:
#usamos la librearia de pandas para extraer csv:
datos_csv =pd.read_csv(r"C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\SELENIUM\sublista_selenium.csv") 

In [4]:
#revisamos los datos:
datos_csv 


,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [5]:
#sacamos solo la informacion de los id:
id_ = datos_csv["ID_Pelicula"]

In [6]:
#La lista final obtenida es:
lista_id = id_.to_list()
print(id_)


0       tt1800301
1       tt1847719
2      tt10270648
3      tt14991492
4       tt2363363
          ...    
995    tt10106152
996    tt10363182
997     tt3857098
998    tt10095498
999     tt1067107
Name: ID_Pelicula, Length: 1000, dtype: object


Debido al alto numero de peliculas a extraer. Dividimos las extracciones en partes de 100 películas 

In [16]:
# Ultima extracción realizada:
lista_id_parte_3=lista_id[401:501]

== ====================================================================================== 

### EXTRAEMOS PELICULAS:

== ======================================================================================

In [17]:
#UTILIZANDO LA FUNCIÓN, EXTRAEMOS LOS ACTORES DE CADA PELÍCULA.
#La asociamos a cada id_película, a traves de un diccionario:

dic_peli={'id_peli':[], 'actores_10':[]}#creamos diccionario donde asociamos las tuplas de actores a cada id_película
no_peli=[] #en esta lista obtenemos los id_peliculas que han dado allgun problema. (La página no tiene datos o no encuentra la película en IMDB)
for id_peli in tqdm(lista_id_parte_3):  #utilizamos la libreria tqdm para llevar a cabo un seguimiento de la extracción.
    try:
        tupla_una_peli=tuplas_actores(id_peli) #utilizamos la función tupla_actores()
        print(tupla_una_peli) # obtenemos la tupla, para revisar.
        dic_peli['actores_10'].append(tupla_una_peli)#añadimos al diccionario.
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        print('fallo peli:--------->', id_peli)# Si hay un fallo en la extracción, nos imprime el id_pelicula para su comprobación.
        continue

    
print(dic_peli)

  1%|          | 1/100 [12:28<20:34:39, 748.28s/it]

[('Katy Clayton', '1994', ['Reparto', 'Reparto'], ['Me, Myself and Di', 'Casualty', 'Shameless'], 'NULL'), ('Russell Richardson', 'NULL', ['Reparto', 'Reparto'], ['Me, Myself and Di', 'Las crónicas del miedo', 'La ciudad y la ciudad'], 'NULL'), ('Chrissie Cotterill', '1955', ['Reparto', 'Reparto'], ['Secretos de familia', 'May to December', 'Doctor Who'], 'NULL'), ('Lucy Pinder', '1983', ['Reparto', 'Guion', 'Equipos adicionales'], ['Strippers vs Werewolves', 'Warrior Savitri', 'LiveJustine'], 'NULL'), ('Adonis Siddique', 'NULL', ['Reparto', 'Reparto'], ['Tin Star', 'Tyrant', 'Newark, Newark'], 'NULL'), ('Perry Benson', '1961', ['Reparto', 'Guion', 'Producción'], ['Sid y Nancy', 'This Is England', "You Rang, M'Lord?"], 'NULL'), ('Wim Snape', '1985', ['Reparto', 'Reparto'], ['Full Monty', 'Emmerdale', 'Genesis'], 'NULL'), ('Tina Louise Owens', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Morbius', 'Supernova', 'Me, Myself and Di'], 'NULL'), ('Simon Gemmell', 'NULL', ['Repart

  2%|▏         | 2/100 [24:24<19:51:13, 729.32s/it]

[('Benoît Magimel', '1974', ['Reparto', 'Reparto'], ['La pianista', 'Los ladrones', 'Los ríos de color púrpura 2: Los ángeles del apocalipsis'], ["Acteurs à l'Écran 1997 Ganador Prix Michel Simon Best Actor Los ladrones", 'Cannes Film Festival 2001 Ganador Best Actor La pianista', 'César Awards, France 2023 Ganador César Best Actor (Meilleur acteur) Pacifiction 2022 Ganador César Best Actor (Meilleur acteur) Es su vida 2016 Ganador César Best Supporting Actor (Meilleur acteur dans un second rôle) La cabeza alta 2013 Nominado César Best Supporting Actor (Meilleur acteur dans un second rôle) Cloclo 1997 Nominado César Most Promising Actor (Meilleur jeune espoir masculin) Los ladrones', 'Valenciennes International Festival of Action and Adventure Films 2002 Ganador Rémy Julienne Award', 'Russian Guild of Film Critics 2001 Nominado Golden Aries Best Foreign Actor La pianista']), ('Igor van Dessel', '2003', ['Reparto', 'Reparto'], ['Close', 'Des feux dans la nuit', 'Dune Dreams'], ['Brecken

  3%|▎         | 3/100 [37:47<20:33:25, 762.95s/it]

...

[('Tisa Skabar', 'NULL', ['Reparto', 'Reparto'], ['Koncert', 'Tiho'], 'NULL'), ('Neca Jazbec', 'NULL', ['Reparto', 'Reparto'], ['Koncert'], 'NULL'), ('Marusa Majer', 'NULL', ['Reparto', 'Reparto'], ['Ivan', 'A'], ['Berlin International Film Festival 2017 Ganador EFP Shooting Star Slovenia.', 'Pula Film Festival 2018 Ganador Golden Arena Best Acting, Croatian Minority Coproductions Ivan', 'Slovene Film Festival 2017 Ganador Vesna Best Actress Ivan', 'Avanca Film Festival 2018 Ganador Best Acting Best Actress Ivan', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Lotos Sparovec', '1971', ['Reparto', 'Reparto'], ['Idila', 'Pojdi z mano', 'Mame'], 'NULL'), ('Janja Majzelj', 'NULL', ['Reparto', 'Reparto'], ['Rusevine', 'Panika', "A Midsummer Night's Dream"], 'NULL'), ('Jus Golmajer', 'NULL', ['Reparto', 'Reparto'], ['Koncert', 'Tiho'], 'NULL')]


 99%|█████████▉| 99/100 [12:45:16<04:34, 274.55s/it]

[]


100%|██████████| 100/100 [12:54:07<00:00, 464.48s/it]

[('Osman Cavci', '1962', ['Reparto', 'Guion', 'Dirección'], ['Zampara Seyfettin 2', 'Otel Prozak', 'En Kahraman Osman'], 'NULL'), ('Aksel Yakup Bektas', 'NULL', ['Reparto', 'Edición', 'Dirección'], ['Karanliktaki Kadin', 'Dünyanin sonunun geldigi gün', 'Kasimpasa Canavari'], 'NULL'), ('Hazal Uzunoglu', 'NULL', ['Reparto', 'Dirección', 'Reparto'], ['Karanliktaki Kadin', 'Dünyanin sonunun geldigi gün', 'DSGG 2: Karantina'], 'NULL'), ('Cenan Gökçeoglu', 'NULL', ['Reparto', 'Reparto'], ['Dünyanin sonunun geldigi gün', 'Kaybettigimiz Seyler'], 'NULL'), ('Oguzhan Yamali', 'NULL', ['Reparto', 'Reparto'], ['Dünyanin sonunun geldigi gün', 'DSGG 2: Karantina'], 'NULL'), ('Miray Cakir', 'NULL', ['Reparto', 'Reparto'], ['Dünyanin sonunun geldigi gün'], 'NULL'), ('Ferdi Döngel', 'NULL', ['Reparto', 'Reparto'], ['Dünyanin sonunun geldigi gün'], 'NULL')]
{'id_peli': ['tt10565666', 'tt11018038', 'tt15010292', 'tt10178280', 'tt1633191', 'tt11197496', 'tt10015518', 'tt14838060', 'tt10101588', 'tt2402157

In [18]:
#con la librería d epandas los pasamos todo a un dataFrame y guardamos en un csv.
df_todo=pd.DataFrame(dic_peli)
df_todo
#convierto en cvs 
df_todo.to_csv('actores_pelis_400_500.csv', index=False)